In [12]:
from numpy import mean
from numpy import std
from numpy import dstack
import numpy as np
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from sklearn.metrics import classification_report

In [13]:

# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None)
    return dataframe.values

In [14]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [15]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'demo/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'demo/')
    print(testX.shape, testy.shape)
    # zero-offset class values
#     trainy = trainy - 1
#     testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [16]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    # 	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
#     filenames += ['body_acc_x_'+group+'.txt']
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [17]:
def create_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print("timestep: ",n_timesteps, "n_features: ",n_features)
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(20, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
#     y_pred = model.predict(testX, batch_size=64, verbose=1)
#     y_pred_bool = np.argmax(y_pred, axis=1)
#     print(testy)
#     print(y_pred_bool)
#     print(classification_report(testy, y_pred_bool))
    # evaluate model
#     _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return model

In [18]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(20, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
#     y_pred = model.predict(testX, batch_size=64, verbose=1)
#     y_pred_bool = np.argmax(y_pred, axis=1)
#     print(testy)
#     print(y_pred_bool)
#     print(classification_report(testy, y_pred_bool))
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [19]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [20]:
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [21]:
def saveModel():
    trainX, trainy, testX, testy = load_dataset()
    model = create_model(trainX, trainy, testX, testy)
    model.save('./modelTest')

In [22]:
run_experiment()

(1100, 20, 6) (1100, 1)
(268, 20, 6) (268, 1)
(1100, 20, 6) (1100, 3) (268, 20, 6) (268, 3)
>#1: 76.866
>#2: 81.343
>#3: 81.343
>#4: 85.448
>#5: 85.821
>#6: 80.970
>#7: 87.687
>#8: 81.343
>#9: 79.104
>#10: 85.448
[76.86567306518555, 81.34328126907349, 81.34328126907349, 85.447758436203, 85.8208954334259, 80.97015023231506, 87.68656849861145, 81.34328126907349, 79.10447716712952, 85.447758436203]
Accuracy: 82.537% (+/-3.239)


In [25]:
saveModel()

(1100, 20, 6) (1100, 1)
(268, 20, 6) (268, 1)
(1100, 20, 6) (1100, 3) (268, 20, 6) (268, 3)
timestep:  20 n_features:  6
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./modelTest\assets


In [26]:
from tensorflow import keras
model2 = keras.models.load_model('./modelTest')

In [27]:
trainX, trainy, testX, testy = load_dataset()

(1100, 20, 6) (1100, 1)
(268, 20, 6) (268, 1)
(1100, 20, 6) (1100, 3) (268, 20, 6) (268, 3)


In [28]:

for item in testX:
    result = model2.predict(np.expand_dims(item, axis=0))
#     print(item)
#     print(result)
    y = np.argmax(result, axis=-1)
    print(y)

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]


In [81]:
print(testX.shape)

(167, 10, 6)


In [83]:
print(testy)
y = np.argmax(testy, axis=-1)
print(y)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0.

In [88]:
print(testX[:2])

[[[ 7.1000e-01 -0.0000e+00 -4.8000e-01  1.1568e+02  9.9550e+01
   -1.1691e+02]
  [ 6.6000e-01 -2.8000e-01 -8.5000e-01  2.0120e+01  6.7490e+01
   -8.1460e+01]
  [ 3.2000e-01 -5.4000e-01 -1.0500e+00 -5.1020e+01 -2.3100e+00
   -1.2130e+01]
  [ 4.0000e-01  4.9000e-01 -5.8000e-01  1.4490e+01 -3.3210e+01
    1.3490e+01]
  [ 2.2000e-01 -6.4000e-01 -8.3000e-01 -8.5100e+00 -1.1450e+01
    2.3580e+01]
  [ 3.0000e-01 -6.1000e-01 -8.2000e-01 -2.1810e+01  4.1500e+00
    1.0450e+01]
  [ 3.0000e-01 -6.0000e-01 -8.1000e-01 -1.0930e+01  3.3000e-01
   -3.2600e+00]
  [ 3.0000e-01 -5.7000e-01 -7.4000e-01 -1.2685e+02  3.3100e+01
    7.0140e+01]
  [ 7.0000e-02 -5.4000e-01 -6.5000e-01  1.4316e+02 -8.2500e+01
   -3.2190e+01]
  [ 3.3000e-01 -5.2000e-01 -8.8000e-01  5.8290e+01  2.2000e-01
    5.8300e+00]]

 [[ 3.0000e-01 -6.1000e-01 -8.2000e-01 -2.1810e+01  4.1500e+00
    1.0450e+01]
  [ 3.0000e-01 -6.0000e-01 -8.1000e-01 -1.0930e+01  3.3000e-01
   -3.2600e+00]
  [ 3.0000e-01 -5.7000e-01 -7.4000e-01 -1.2685e+02

In [91]:
print(np.expand_dims(testX[0], axis=0))

[[[ 7.1000e-01 -0.0000e+00 -4.8000e-01  1.1568e+02  9.9550e+01
   -1.1691e+02]
  [ 6.6000e-01 -2.8000e-01 -8.5000e-01  2.0120e+01  6.7490e+01
   -8.1460e+01]
  [ 3.2000e-01 -5.4000e-01 -1.0500e+00 -5.1020e+01 -2.3100e+00
   -1.2130e+01]
  [ 4.0000e-01  4.9000e-01 -5.8000e-01  1.4490e+01 -3.3210e+01
    1.3490e+01]
  [ 2.2000e-01 -6.4000e-01 -8.3000e-01 -8.5100e+00 -1.1450e+01
    2.3580e+01]
  [ 3.0000e-01 -6.1000e-01 -8.2000e-01 -2.1810e+01  4.1500e+00
    1.0450e+01]
  [ 3.0000e-01 -6.0000e-01 -8.1000e-01 -1.0930e+01  3.3000e-01
   -3.2600e+00]
  [ 3.0000e-01 -5.7000e-01 -7.4000e-01 -1.2685e+02  3.3100e+01
    7.0140e+01]
  [ 7.0000e-02 -5.4000e-01 -6.5000e-01  1.4316e+02 -8.2500e+01
   -3.2190e+01]
  [ 3.3000e-01 -5.2000e-01 -8.8000e-01  5.8290e+01  2.2000e-01
    5.8300e+00]]]


In [102]:
result = model2.predict([[[ 7.1000e-01, -0.0000e+00 ,-4.8000e-01 , 1.1568e+02 , 9.9550e+01,
   -1.1691e+02],
  [ 6.6000e-01 ,-2.8000e-01 ,-8.5000e-01 , 2.0120e+01 , 6.7490e+01, -8.1460e+01],
  [ 3.2000e-01, -5.4000e-01 ,-1.0500e+00, -5.1020e+01,-2.3100e+00,-1.2130e+01],
  [ 4.0000e-01 , 4.9000e-01, -5.8000e-01 , 1.4490e+01, -3.3210e+01,
    1.3490e+01],
  [ 2.2000e-01, -6.4000e-01 ,-8.3000e-01 ,-8.5100e+00, -1.1450e+01,
    2.3580e+01],
  [ 3.0000e-01, -6.1000e-01, -8.2000e-01,-2.1810e+01,  4.1500e+00,
    1.0450e+01],
  [ 3.0000e-01, -6.0000e-01, -8.1000e-01, -1.0930e+01,  3.3000e-01,
   -3.2600e+00],
  [ 3.0000e-01, -5.7000e-01 ,-7.4000e-01, -1.2685e+02 , 3.3100e+01,
    7.0140e+01],
  [ 7.0000e-02 ,-5.4000e-01, -6.5000e-01 , 1.4316e+02,-8.2500e+01,
   -3.2190e+01],
  [ 3.3000e-01, -5.2000e-01, -8.8000e-01,  5.8290e+01 , 2.2000e-01,
    5.8300e+00]]])
print(result)
y = np.argmax(result, axis=-1)
print(y)

[[9.9854666e-01 1.4209755e-12 1.4532653e-03]]
[0]


In [104]:
print(testX[0])

[[ 7.1000e-01 -0.0000e+00 -4.8000e-01  1.1568e+02  9.9550e+01 -1.1691e+02]
 [ 6.6000e-01 -2.8000e-01 -8.5000e-01  2.0120e+01  6.7490e+01 -8.1460e+01]
 [ 3.2000e-01 -5.4000e-01 -1.0500e+00 -5.1020e+01 -2.3100e+00 -1.2130e+01]
 [ 4.0000e-01  4.9000e-01 -5.8000e-01  1.4490e+01 -3.3210e+01  1.3490e+01]
 [ 2.2000e-01 -6.4000e-01 -8.3000e-01 -8.5100e+00 -1.1450e+01  2.3580e+01]
 [ 3.0000e-01 -6.1000e-01 -8.2000e-01 -2.1810e+01  4.1500e+00  1.0450e+01]
 [ 3.0000e-01 -6.0000e-01 -8.1000e-01 -1.0930e+01  3.3000e-01 -3.2600e+00]
 [ 3.0000e-01 -5.7000e-01 -7.4000e-01 -1.2685e+02  3.3100e+01  7.0140e+01]
 [ 7.0000e-02 -5.4000e-01 -6.5000e-01  1.4316e+02 -8.2500e+01 -3.2190e+01]
 [ 3.3000e-01 -5.2000e-01 -8.8000e-01  5.8290e+01  2.2000e-01  5.8300e+00]]


In [ ]:
trainX, trainy, testX, testy = load_dataset()
